In [53]:
from dotenv import load_dotenv
import os
import numpy as np 
import pandas as pd
import pdfplumber

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool


load_dotenv()

True

In [54]:
# LLM definition
model = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [55]:
cust_data = {
  "id": 5757,
  "template": "A",
  "image": "",
  "language_selected": "Bahasa Malaysia",
  "picture": "",
  "name": "Nadira Aisyah",
  "adress": "No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur",
  "email": "nadira.aisyah@techmail.com",
  "telephone": "+60 12 345 6789",
  "title": "Pembangun Web Asas",
  "about": "Pembangun web dengan sedikit pengalaman dalam projek-projek asas dan tugasan berkumpulan.",
  "experience": "Company FusionTech Solutions, Jurutera Perisian, Kuala Lumpur, 2021 – Kini. Menyokong pasukan dalam tugas pembangunan harian, membantu membetulkan isu kecil dalam aplikasi web, menghadiri mesyuarat mingguan dan membuat dokumentasi asas projek. Company InnovaCode, Pembangun Web Junior, Selangor, 2019 – 2021. Membantu dalam mereka bentuk halaman web asas, membuat kemas kini kandungan laman web secara berkala, menjalankan ujian ringkas sebelum penghantaran. Company DataMinds, Intern Analisis Data, Cyberjaya, 2018 – 2019. Membantu memindahkan data ke dalam Excel, menyediakan laporan asas dengan bantuan penyelia, mempelajari asas pengendalian dataset kecil.",
  "education": "Ijazah Sarjana Muda Sains Komputer, Universiti Malaya, 2015 – 2019. CGPA 2.80/4.00. STPM Sains, Kolej Tingkatan Enam Cheras, 2013 – 2015. Lulus semua subjek. SPM, SMK Seri Bintang Utara, 2012. 5 kredit termasuk Matematik.",
  "reference": "Pn. Farah Lina, Pegawai Pentadbiran di FusionTech Solutions, farah.lina@fusiontech.com, +60 12 678 9012. En. Ahmad Zulkifli, Pembantu Pensyarah di Universiti Malaya, ahmad.zulkifli@um.edu.my, +60 13 123 4567.",
  "technical_skills": "Python 30%, JavaScript 20%, SQL 15%, React 25%.",
  "soft_skills": "Komunikasi 20%, Kerja Berpasukan 21%, Pengurusan Masa 19%, Kepimpinan 18%.",
  "certification": "Asas Python - 2023, Pengenalan Cloud - 2022."
}

# Data Transformation

In [56]:
# image
# file name
#template 

id_data = cust_data["id"]
template_data = cust_data["template"]
language_data = cust_data["language_selected"]
image_data = cust_data["image"]

cust_data

{'id': 5757,
 'template': 'A',
 'image': '',
 'language_selected': 'Bahasa Malaysia',
 'picture': '',
 'name': 'Nadira Aisyah',
 'adress': 'No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur',
 'email': 'nadira.aisyah@techmail.com',
 'telephone': '+60 12 345 6789',
 'title': 'Pembangun Web Asas',
 'about': 'Pembangun web dengan sedikit pengalaman dalam projek-projek asas dan tugasan berkumpulan.',
 'experience': 'Company FusionTech Solutions, Jurutera Perisian, Kuala Lumpur, 2021 – Kini. Menyokong pasukan dalam tugas pembangunan harian, membantu membetulkan isu kecil dalam aplikasi web, menghadiri mesyuarat mingguan dan membuat dokumentasi asas projek. Company InnovaCode, Pembangun Web Junior, Selangor, 2019 – 2021. Membantu dalam mereka bentuk halaman web asas, membuat kemas kini kandungan laman web secara berkala, menjalankan ujian ringkas sebelum penghantaran. Company DataMinds, Intern Analisis Data, Cyberjaya, 2018 – 2019. Membantu memindahkan data ke dalam Excel, menyediakan laporan asa

# Passing to LLM

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import json

instructions = "English"

if cust_data.get("language_selected") == "English":
    instructions = """
You are a professional resume assistant. All output must be in English, regardless of the input language.

Your task:
1. Clean and restructure the given JSON object into a clean, well-structured format as described.
2. Parse any concatenated string data into appropriate objects or arrays (e.g., experience, education, skills).
3. Enrich resume content with professional tone, added context, and measurable impact.

Formatting Rules:
- Use the structure shown in JSON Object B.
- Do NOT include inline explanations or metadata. Return **only valid JSON**.
- Do NOT return skill arrays. Instead, return a `skills` object split into `technical skills` and `soft skills`.
- All skill proficiencies must be integers only (no %, no quotes).

Required Structure:
- experience: list of objects with `company`, `title`, `location`, `duration`, and `details` (3 bullet points)
- education: list of objects with `level`, `institution`, `duration`, and `details`
- reference: list of objects with `name`, `position`, `company`, `email`, `telephone`
- skills: object with `technical skills` and `soft skills`, each with skill-proficiency pairs
- certification: list of certification strings

Content Enrichment:
- Expand each experience with 3 concise, professional bullet points (~30 words each) including action verbs and quantifiable outcomes.
- Update the "about" section to match the role, add professional context, and keep around 20 words.
- Use polished, action-oriented language throughout.
"""
else:
    instructions = """
Anda adalah pembantu resume profesional. Semua output mesti dalam Bahasa Melayu.

Tugasan anda:
1. Susun semula objek JSON supaya lebih tersusun dan mudah dibaca.
2. Tukar maklumat bercantum kepada format objek atau senarai mengikut struktur yang ditetapkan.
3. Naiktaraf kandungan resume dengan gaya profesional, serta tambah konteks dan kesan yang boleh diukur.

Peraturan Format:
- Gunakan struktur seperti JSON Object B.
- Jangan berikan sebarang penjelasan. Hanya keluarkan JSON yang sah.
- Jangan guna senarai kemahiran. Guna objek `skills` dengan `technical skills` dan `soft skills`.
- Semua kemahiran mesti dinyatakan dalam bentuk integer sahaja (tanpa %, tanpa tanda petik).

Struktur Diperlukan:
- experience: senarai objek dengan `company`, `title`, `location`, `duration`, dan `details` (3 poin ringkas)
- education: senarai objek dengan `level`, `institution`, `duration`, dan `details`
- reference: senarai objek dengan `name`, `position`, `company`, `email`, `telephone`
- skills: objek dengan `technical skills` dan `soft skills`, dengan pasangan kemahiran dan tahap
- certification: senarai sijil dalam bentuk string

Naiktaraf Kandungan:
- Setiap pengalaman kerja perlu mengandungi 3 poin ringkas (~30 patah perkataan), dengan hasil yang boleh diukur.
- Perbaharui bahagian "about" supaya padat, sesuai dengan jawatan, dan sekitar 20 patah perkataan.
- Guna bahasa profesional yang menonjolkan pencapaian.
"""

prompt_template = """
You are a professional assistant that structures and enhances resume details in JSON format.
You will receive a raw JSON object representing a resume and instructions.
Return only the enhanced and properly structured JSON object. No explanation.

Instructions:
{instructions}

Resume JSON:
{resume_json}
"""


resume_json_str = json.dumps(cust_data)
prompt = prompt_template.format(instructions=instructions, resume_json=resume_json_str)
message = HumanMessage(content=prompt)
response = model.invoke([message])

# Cleanup
response_str = response.content
def extract_json_from_response(response_str: str) -> str:
    if response_str.startswith("```json"):
        response_str = response_str[len("```json"):].strip()
    elif response_str.startswith("```"):
        response_str = response_str[len("```"):].strip()
    if response_str.endswith("```"):
        response_str = response_str[:-3].strip()
    return response_str

clean_json_str = extract_json_from_response(response_str)

try:
    json_object = json.loads(clean_json_str)
    print("✅ Parsed JSON successfully!")
except json.JSONDecodeError as e:
    print("❌ Failed to parse JSON:", e)
    json_object = None

✅ Parsed JSON successfully!


In [58]:
print(json_object)

{'id': 5757, 'template': 'A', 'image': '', 'language_selected': 'Bahasa Malaysia', 'picture': '', 'name': 'Nadira Aisyah', 'address': 'No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur', 'email': 'nadira.aisyah@techmail.com', 'telephone': '+60 12 345 6789', 'title': 'Pembangun Web Asas', 'about': 'Pembangun web yang berpengalaman dalam projek asas dengan kemahiran dalam penyelesaian masalah dan kerja berpasukan yang berkesan.', 'experience': [{'company': 'FusionTech Solutions', 'title': 'Jurutera Perisian', 'location': 'Kuala Lumpur', 'duration': '2021 – Kini', 'details': ['Menyokong pasukan dalam pembangunan aplikasi web, meningkatkan kecekapan proses dengan menyelesaikan isu kecil.', 'Menghasilkan dokumentasi projek yang jelas dan terperinci untuk rujukan masa depan.', 'Menghadiri mesyuarat mingguan untuk membincangkan kemajuan dan cabaran projek.']}, {'company': 'InnovaCode', 'title': 'Pembangun Web Junior', 'location': 'Selangor', 'duration': '2019 – 2021', 'details': ['Mereka bentuk da

In [59]:

if language_data == "English": 
    language_selected = "templates English"
else:
    language_selected = "templates Bahasa Malaysia"


if template_data == "A": 
    template_selected = "template_A.html"
elif template_data == "B":
    template_selected = "template_B.html"


In [60]:
from jinja2 import Environment, FileSystemLoader

# Set up Jinja2 environment
env = Environment(loader=FileSystemLoader("."))  # current directory

# direct to temp selected
template = env.get_template(f"./{language_selected}/{template_selected}")

# Render with data
output_html = template.render(**json_object)

# Write to output file
with open(f"./generated html/{id_data}_output_resume.html", "w", encoding="utf-8") as f:
    f.write(output_html)